<a href="https://colab.research.google.com/github/Samin-Sadaf7/Image_works/blob/main/VideoToImageProcessingWebCrawlingForDatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [24]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install pytube opencv-python torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


In [3]:
import os
import cv2
import torch
import pytube
from pytube import YouTube
import matplotlib.pyplot as plt

In [4]:
VIDEO_DIR = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_videos"
FRAME_DIR = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames"
os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(FRAME_DIR, exist_ok=True)

In [17]:
!pip --quiet install pytubefix

In [28]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
def download_youtube_video(url, save_path=VIDEO_DIR):
  yt = YouTube(url, on_progress_callback=on_progress)
  video_title = yt.title
  print(f"Downloading video: {yt.title}")
  ys = yt.streams.get_highest_resolution()
  video_path = ys.download(output_path=VIDEO_DIR)
  print(f"Video downloaded to: {video_path}")
  return video_path, video_title

In [56]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
print(model)

100%|██████████| 21.5M/21.5M [00:00<00:00, 34.1MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [99]:
#Real Time Detection transformer
from ultralytics import RTDETR
model = RTDETR('rtdetr-l.pt')
print(model)

100%|██████████| 63.4M/63.4M [00:01<00:00, 45.2MB/s]


RTDETR(
  (model): DetectionModel(
    (model): Sequential(
      (0): HGStem(
        (stem1): Conv(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem2a): Conv(
          (conv): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem2b): Conv(
          (conv): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem3): Conv(
          (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, mome

In [100]:
labels = ["stop sign", "traffic sign", "speed limit"]

In [101]:
def extract_frames_with_traffic_signs(video_path, video_title, output_dir=FRAME_DIR, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    frame_number = 0
    extracted_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Only process frames at specified intervals
        if frame_number % frame_interval == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = model.predict(rgb_frame)

            # YOLOv8 results are stored in results[0].boxes
            boxes = results[0].boxes

            for box in boxes:
                # Get bounding box coordinates and class
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                cls = int(box.cls[0])
                label = model.names[cls]

                if label in labels:
                    timestamp = frame_number / frame_rate
                    timestamp_str = f"{timestamp:.2f}s"

                    sanitized_title = video_title.replace(" ", "_").replace("/", "_")
                    frame_filename = os.path.join(output_dir, f"{sanitized_title}_frame_{timestamp_str}.jpg")

                    extracted_count += 1
                    cv2.imwrite(frame_filename, frame)
                    print(f"Traffic sign detected and frame saved: {frame_filename}")

        frame_number += 1

    cap.release()
    cv2.destroyAllWindows()

In [103]:
video_urls = [
  "https://www.youtube.com/watch?v=wEkfibQQPF0",
  "https://www.youtube.com/watch?v=dvjGIDgeUkE",
  "https://www.youtube.com/watch?v=1NgUrKP5GTY",
]

for url in video_urls:
      video_path, video_title = download_youtube_video(url)
      extract_frames_with_traffic_signs(video_path, video_title)

Streaming output truncated to the last 5000 lines.
Speed: 7.6ms preprocess, 2123.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 2 bicycles, 1 motorcycle, 2 trucks, 1 traffic light, 2163.9ms
Speed: 6.5ms preprocess, 2163.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 5 motorcycles, 2 trucks, 2176.3ms
Speed: 7.1ms preprocess, 2176.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1 motorcycle, 1 truck, 3067.6ms
Speed: 5.7ms preprocess, 3067.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 3 motorcycles, 1 truck, 3171.9ms
Speed: 4.7ms preprocess, 3171.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 cars, 2 motorcycles, 1 bus, 1 truck, 2176.3ms
Speed: 8.1ms preprocess, 2176.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 motorcycle, 1 bus, 2192.9ms

In [37]:
import os
import shutil

def empty_directory(directory_path):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        print("Directory does not exist or is not a directory.")

In [49]:
directory_to_empty = FRAME_DIR
empty_directory(directory_to_empty)

In [97]:
directory_to_empty = VIDEO_DIR
empty_directory(directory_to_empty)

Web Scrapping

In [65]:
!pip install google_images_download

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for google_images_download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14537 sha256=d80b2be9b9a3d7455ae212041e57349573fb6b6c60c28ac5503dee0e41bb840c
  Stored in directory: /root/.cache/pip/wheels/5b/f2/64/0eecb22fee5cbc0321d332492ddcc45b03c6373b5616ee95c8
Successfully built google_images_download


In [75]:
!pip install icrawler

In [77]:
from icrawler.builtin import GoogleImageCrawler
import os

In [91]:
traffic_signs = [
    "Bangladesh Speed Limit 60 km/h sign on road in Bangladesh",
    "Bangladesh School traffic sign on road in Bangladesh",
    "Bangladesh Pedestrian Crossing traffic sign on road in Bangladesh",
    "Bangladesh Sharp Bend To The Left traffic sign on road in Bangladesh",
    "Bangladesh U-Turn traffic sign on road in Bangladesh",
    "Bangladesh Speed Limit 80 km/h sign on road in Bangladesh",
    "Bangladesh No Overtaking traffic sign on road in Bangladesh",
    "Bangladesh Sharp Bend To The Right traffic sign on road in Bangladesh",
    "Bangladesh Side Road Left traffic sign on road in Bangladesh",
    "Bangladesh Side Road Right traffic sign on road in Bangladesh",
    "Bangladesh Speed Limit 40 km/h sign on road in Bangladesh",
    "Bangladesh Narrow Bridge traffic sign on road in Bangladesh",
    "Bangladesh Road Hump traffic sign on road in Bangladesh",
    "Bangladesh No Use of Horn traffic sign on road in Bangladesh",
    "Bangladesh Crossroads traffic sign on road in Bangladesh"
]

In [87]:
# Define the base output directory for saving images
base_output_dir = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images"

In [92]:
def download_traffic_sign_images(signs, limit=5):
    # Ensure the base directory exists
    if not os.path.exists(base_output_dir):
        os.makedirs(base_output_dir)

    for sign in signs:
        # Sanitize folder name
        folder_name = sign.replace(" ", "_").replace("/", "_")
        sign_output_dir = os.path.join(base_output_dir, folder_name)

        # Ensure each sign's folder exists
        if not os.path.exists(sign_output_dir):
            os.makedirs(sign_output_dir)

        # Initialize the Google Image Crawler
        google_crawler = GoogleImageCrawler(storage={'root_dir': sign_output_dir})

        # Start the crawl
        google_crawler.crawl(keyword=sign, max_num=limit, filters=None, file_idx_offset='auto')
        print(f"Downloaded images for {sign} and saved in {sign_output_dir}")

In [94]:
# Run the download function for the traffic signs
download_traffic_sign_images(traffic_signs)

ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/976242716/photo/speed-limit-sign-60-km-h.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1150813637/photo/60-kilometres-per-hour-speed-limit-road-warning-sign.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/465649993/vector/german-road-sign-speed-limit-60-kmh.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1481245118/photo/road-sign.jpg


Downloaded images for Bangladesh Speed Limit 60 km/h sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Speed_Limit_60_km_h_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg
ERROR:downloader:Response status code 403, file https://www.researchgate.net/publication/4376301/figure/fig1/AS:279955654103061@1443758037299/Standard-road-signs-with-Bangla-script-panel.png
ERROR:downloader:Response status code 403, file https://www.researchgate.net/profile/Mahmudul-Hasan-60/publication/224157461/figure/fig1/AS:452634499915781@1484927882722/Some-of-the-Mandatory-traffic-signs-in-Bangladesh_Q320.jpg


Downloaded images for Bangladesh School traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_School_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77fcef50a8f0>: Failed to resolve 'w1.png' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77fcef50aa40>: Failed to resolve 'w1.png' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by Name

...

Downloaded images for Bangladesh Road Hump traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Road_Hump_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg


Downloaded images for Bangladesh No Use of Horn traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_No_Use_of_Horn_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 403, file https://www.researchgate.net/publication/381608451/figure/fig2/AS:11431281254030562@1719027721676/Various-Types-of-Bangladesh-Traffic-Signs.jpg


Downloaded images for Bangladesh Crossroads traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Crossroads_traffic_sign_on_road_in_Bangladesh
